# Combustion

Examining combustion problems (and chemical equilibrium problems) generally requires calculating thermodynamic properties of chemical species. 

Cantera makes it easy to both work with mixtures and pure components, and determine their thermodynamic properties, but it requires you to first load a model containing some model for these (typically represented as a set of coefficients for a high-order polynomial).

## Built-in models

Fortunately, Cantera comes with a few built-in models meant to be used for air, hydrogen/oxygen combustion, and natural gas combustion, which provide quick access to properties for many species:

In [9]:
from textwrap import wrap
import cantera as ct

In [17]:
# Models built into Cantera with commonly used gas species

models = ['air.cti', 'h2o2.cti', 'gri30.cti']

for model in models:
    gas = ct.Solution(model)
    
    print(f"Species in '{model}' model:")
    lines = wrap(' '.join(gas.species_names), width=60, break_long_words=False)
    for line in lines:
        print(line)
    print()

Species in 'air.cti' model:
O O2 N NO NO2 N2O N2 AR

Species in 'h2o2.cti' model:
H2 H O O2 OH H2O HO2 H2O2 AR

Species in 'gri30.cti' model:
H2 H O O2 OH H2O HO2 H2O2 C CH CH2 CH2(S) CH3 CH4 CO CO2 HCO
CH2O CH2OH CH3O CH3OH C2H C2H2 C2H3 C2H4 C2H5 C2H6 HCCO
CH2CO HCCOH N NH NH2 NH3 NNH NO NO2 N2O HNO CN HCN H2CN HCNN
HCNO HOCN HNCO NCO N2 AR C3H7 C3H8 CH2CHO CH3CHO



We will frequently use the `gri30.cti` model, which is the GRI-Mech 3.0 model originally developed to model natural gas combustion {cite}`gri30`.

Cantera also comes with a large database of thermodynamic properties for gases, which originally came from NASA {cite}`Gordon1971,McBride1993`. 

Due to the large number of components present, and the sometimes-unusual naming, it may be a bit challenging to easily find a compound you are looking for; the easiest way may be to search the [full file available online](https://github.com/Cantera/cantera/blob/master/data/inputs/nasa_gas.cti).

In [28]:
species = {S.name: S for S in ct.Species.listFromFile('nasa_gas.cti')}

print('Number of species: ', len(species))

Number of species:  748


Let's say that we want to consider the complete combustion of gasoline in air. Gasoline is a very complex mixture of large hydrocarbons, without a fixed composition, but we can model it as a mixture of _n_-heptane (C$_{7}$H$_{16}$) and isooctane (C$_{8}$H$_{18}$). (These actually represent 0 and 100 on the octane rating.)

After searching the NASA database, we find these species as `C7H16,n-heptane` and `C8H18,isooctane`, so we can extract them, the components of air, and the complete combustion products:

In [27]:
complete_species = [species[S] for S in (
    'C7H16,n-heptane', 'C8H18,isooctane', 'O2', 'N2', 'CO2', 'H2O'
    )]
gas = ct.Solution(thermo='IdealGas', species=complete_species)

Then, we can specify a stoichiometric mixture of 87% isooctane \ 13% _n_-heptane by mole, at standard conditions:

In [29]:
gas.TP = 298.15, ct.one_atm
gas.set_equivalence_ratio(
    1.0, 'C7H16,n-heptane: 0.13, C8H18,isooctane: 0.87',
    'O2: 1.0, N2: 3.76'
    )
gas()


       temperature          298.15  K
          pressure          101325  Pa
           density         1.23658  kg/m^3
  mean mol. weight         30.2534  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy     -1.2168e+05       -3.681e+06     J
   internal energy     -2.0362e+05        -6.16e+06     J
           entropy          6718.8        2.033e+05     J/K
    Gibbs function     -2.1249e+06       -6.429e+07     J
 heat capacity c_p          1051.1         3.18e+04     J/K
 heat capacity c_v          776.29        2.349e+04     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
   C7H16,n-heptane     0.00218224       0.00722794         -133.372
   C8H18,isooctane      0.0146042        0.0551428         -145.476
                O2       0.206557         0.218475         -26.2508
                N2       0.776656   

Then, we can determine the equilibrium composition and adiabatic flame temperature, after complete combustion:

In [30]:
gas.equilibrate('HP')
gas()


       temperature         2403.07  K
          pressure          101325  Pa
           density        0.145059  kg/m^3
  mean mol. weight          28.604  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy     -1.2168e+05       -3.481e+06     J
   internal energy     -8.2019e+05       -2.346e+07     J
           entropy          9614.1         2.75e+05     J/K
    Gibbs function     -2.3225e+07       -6.643e+08     J
 heat capacity c_p          1474.8        4.219e+04     J/K
 heat capacity c_v          1184.2        3.387e+04     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
   C7H16,n-heptane    3.04892e-12      1.06808e-11         -131.471
                O2    3.35656e-11      3.75492e-11         -53.5511
                N2       0.734314         0.719155         -27.8845
               CO2       0.124907   

## Finding other models

You might face a problem that requires thermodynamic properties for compounds not available in the built-in models. In that case, you might need to find an entire model that contains the compound(s) of interest, or at least the thermodynamic parameters for each species.

You may find models for particular systems by searching the combustion/thermodynamics literature (i.e., Google Scholar). These models, either available as supplementary material for a journal article or via a research group's website, typically come in the older Chemkin format, which needs to be [converted to Cantera's CTI format](https://cantera.org/tutorials/ck2cti-tutorial.html).

Another powerful source of thermodynamics information is the Reaction Mechanism Generator (RMG) [Molecule Search](https://rmg.mit.edu/molecule_search) {cite}`Allen2012,Magoon2013,Gao2016`, which offers the ability to both search for information in its databases but also estimate properties for any molecule. 

For example, say we want to calculate thermodynamic properties for toluene (C$_6$H$_5$CH$_3$, or C$_7$H$_8$). You can go to the [RMG Molecule Search](https://rmg.mit.edu/molecule_search), enter `toluene` in the "Species Identifier" field and hit tab. Then, after the site responds, click "Search Thermochemistry". 
This brings up multiple entries for this molecule. 

Let's use the "USC-Mech_ii" result. This includes the CHEMKIN format NASA Polynomial:
```
! USC-Mech-ii
C7H8                    C   7H   8          G   100.000  5000.000 1073.72      1
 9.30667130E+00 3.36713230E-02-1.41158452E-05 2.69973752E-09-1.93204076E-13    2
 5.79813528E+02-2.68610894E+01 2.05719474E+00 3.12152776E-02 3.04758250E-05    3
-5.05435593E-08 1.81542493E-11 3.83493690E+03 1.65378111E+01                   4
```
which contains the elemental composition (7 C and 8 H), the phase (`G`: gas), the temperature ranges for applicability of this model, and 14 polynomials used to obtain the standard-state specific heat, enthalpy, and entropy as a function of temperature. The first seven coefficients are for the upper temperature range (specified by the middle and high temperatures, 1073.72 and 5000 K here), and the next seven coefficients are for the lower temperature range (100 to 1073.72 K here).

We can take this information and create a Cantera `Species` object, with the name `C6H5CH3`:

In [32]:
species_toluene = ct.Species.fromCti(
    '''species(name='C6H5CH3', atoms='C:7,H:8',
    thermo=(NASA([100.00, 1073.72],
                 [ 2.05719474E+00,  3.12152776E-02,  3.04758250E-05,
                  -5.05435593E-08,  1.81542493E-11,  3.83493690E+03,
                   1.65378111E+01]),
            NASA([1073.72, 5000.00],
                 [ 9.30667130E+00,  3.36713230E-02,  -1.41158452E-05,
                   2.69973752E-09, -1.93204076E-13,   5.79813528E+02,
                  -2.68610894E+01])))'''
        )

Then, we can create a new model that contains toluene (`C6H5CH3`), as well as the components of air and the complete combustion products:

In [34]:
species = {S.name: S for S in ct.Species.listFromFile('gri30.cti')}
complete_species = [species[S] for S in ('O2', 'N2', 'CO2', 'H2O')]
complete_species += [species_toluene]
                                         
gas = ct.Solution(thermo='IdealGas', species=complete_species)

gas.TP = 298.15, ct.one_atm
gas.set_equivalence_ratio(1.0, 'C6H5CH3: 1.0', 'O2: 1.0, N2: 3.76')
gas.equilibrate('HP')
gas()


       temperature         2502.69  K
          pressure          101325  Pa
           density        0.144226  kg/m^3
  mean mol. weight         29.6188  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy           37684        1.116e+06     J
   internal energy     -6.6486e+05       -1.969e+07     J
           entropy          9365.7        2.774e+05     J/K
    Gibbs function     -2.3402e+07       -6.931e+08     J
 heat capacity c_p          1422.6        4.214e+04     J/K
 heat capacity c_v          1141.9        3.382e+04     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
                O2     6.9353e-10      7.49259e-10         -50.6768
                N2       0.754683         0.713781         -27.9926
               CO2       0.156111         0.231961         -53.7422
               H2O      0.0892061   